### 주택별 세대 수 파악
- 해당하는 데이터보다 더 좋은 데이터를 찾아서 미사용

In [2]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from bs4 import BeautifulSoup
import json
import xmltodict
import re
import pickle

In [4]:
'''
# 아파트 단지 코드 불러오기
# Service URL
xmlUrl = 'https://apis.data.go.kr/1613000/AptListService2/getSidoAptList'

My_API_Key = unquote('IzZ1ERXpwQ%2BahxPK9xPLgMYipy4G3SkIRRj4%2F8ITTaLJGKwRWdlqjOt77SIAAZvKytmH5zFPZyAdNvZFddlwZg%3D%3D')    
queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : My_API_Key,    # 서비스키
        quote_plus('sidoCode') : '11',          # 시도번호
        quote_plus('pageNo') : '1',        # 페이지 번호
        quote_plus('numOfRows') : '3000'        # 게시물 번호
     }
)

response = requests.get(xmlUrl + queryParams, verify=False).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
rows = xmlobj.findAll('item')
rows
'''

/opt/homebrew/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[<item><as1>서울특별시</as1><as2>금천구</as2><as3>시흥동</as3><bjdCode>1154510300</bjdCode><kaptCode>A15303103</kaptCode><kaptName>시흥성지</kaptName></item>,
 <item><as1>서울특별시</as1><as2>강남구</as2><as3>도곡동</as3><bjdCode>1168011800</bjdCode><kaptCode>A13527007</kaptCode><kaptName>도곡1차아이파크</kaptName></item>,
 <item><as1>서울특별시</as1><as2>양천구</as2><as3>신정동</as3><bjdCode>1147010100</bjdCode><kaptCode>A15886402</kaptCode><kaptName>신정대성유니드아파트2차</kaptName></item>,
 <item><as1>서울특별시</as1><as2>노원구</as2><as3>중계동</as3><bjdCode>1135010600</bjdCode><kaptCode>A13986111</kaptCode><kaptName>중계라이프신동아청구아파트</kaptName></item>,
 <item><as1>서울특별시</as1><as2>노원구</as2><as3>하계동</as3><bjdCode>1135010400</bjdCode><kaptCode>A13923106</kaptCode><kaptName>하계2차현대아파트</kaptName></item>,
 <item><as1>서울특별시</as1><as2>서초구</as2><as3>서초동</as3><bjdCode>1165010800</bjdCode><kaptCode>A13785607</kaptCode><kaptName>롯데캐슬클래식</kaptName></item>,
 <item><as1>서울특별시</as1><as2>강서구</as2><as3>마곡동</as3><bjdCode>1150010500</bjdCode><kaptCode>A15721003</kaptCo

In [5]:
'''
# 아파트 단지 코드 csv파일로 저장
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장 (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. 
    
result = pd.DataFrame(rowList, columns=nameList)
result.to_csv('aptCode.csv', index=False)
'''

In [7]:
'''
# 아파드 단지 정보 Api
# Service URL
xmlUrl = 'http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo'
My_API_Key = unquote('IzZ1ERXpwQ%2BahxPK9xPLgMYipy4G3SkIRRj4%2F8ITTaLJGKwRWdlqjOt77SIAAZvKytmH5zFPZyAdNvZFddlwZg%3D%3D')    
rows = bs4.element.ResultSet([])
for i in code['kaptCode']:
    queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
        {
            quote_plus('ServiceKey') : My_API_Key,    # 서비스키
            quote_plus('kaptCode') : '{}'.format(i)         # 시도번호
        }
    )

    response = requests.get(xmlUrl + queryParams, verify=False).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    rows += xmlobj.findAll('item')
rows
'''

In [12]:
'''
# pKl 파일 저장
with open('filename.pkl', 'wb') as f:
	pickle.dump(rows, f, protocol=pickle.HIGHEST_PROTOCOL)
'''

In [13]:
'''
# pkl 파일 로드
with open('filename.pkl', 'rb') as f:
	data = pickle.load(f)
data
'''

In [35]:
'''
# 원하는 단지정보 저장
rowList = []
columnList = ['bjdCode','codeAptNm','hoCnt','kaptAddr','kaptCode','kaptdaCnt','kaptName','kaptDongCnt']
dataLen = len(data)
for i in range(0,dataLen):
    info = []
    # 법정동 코드
    if data[i].find('bjdCode'):
        info.append(data[i].find('bjdCode').text)
    else:
        info.append('NaN')
    # 단지분류
    if data[i].find('codeAptNm'):
        info.append(data[i].find('codeAptNm').text)
    else:
        info.append('NaN')
    # 호수
    if data[i].find('hoCnt'):
        info.append(data[i].find('hoCnt').text)
    else:
        info.append('NaN')
    # 법정동 주소
    if data[i].find('kaptAddr'):
        info.append(data[i].find('kaptAddr').text)
    else:
        info.append('NaN')
    # 주택코드
    if data[i].find('kaptCode'):
        info.append(data[i].find('kaptCode').text)
    else:
        info.append('NaN')
    # 세대수
    if data[i].find('kaptdaCnt'):
        info.append(data[i].find('kaptdaCnt').text)
    else:
        info.append('NaN')
    # 단지이름
    if data[i].find('kaptName'):
        info.append(data[i].find('kaptName').text)
    else:
        info.append('NaN')
    # 동수
    if data[i].find('kaptDongCnt'):
        info.append(data[i].find('kaptDongCnt').text)
    else:
        info.append('NaN')
    rowList.append(info)
'''

[['1154510300',
  '아파트',
  '233',
  '서울특별시 금천구 시흥동 173-8 시흥성지',
  'A15303103',
  '233',
  '시흥성지',
  '2'],
 ['1168011800',
  '아파트',
  '321',
  '서울특별시 강남구 도곡동 543-7 도곡1차아이파크',
  'A13527007',
  '321',
  '도곡1차아이파크',
  '5'],
 ['1147010100',
  '아파트',
  '186',
  '서울특별시 양천구 신정동 1198-3 신정대성유니드아파트2차',
  'A15886402',
  '186',
  '신정대성유니드아파트2차',
  '2'],
 ['1135010600',
  '아파트',
  '960',
  '서울특별시 노원구 중계동 366 중계라이프신동아청구아파트',
  'A13986111',
  '960',
  '중계라이프신동아청구아파트',
  '17'],
 ['1135010400',
  '아파트',
  '730',
  '서울특별시 노원구 하계동 288 하계2차현대아파트',
  'A13923106',
  '730',
  '하계2차현대아파트',
  '12'],
 ['1165010800',
  '아파트',
  '990',
  '서울특별시 서초구 서초동 1312-3 롯데캐슬클래식',
  'A13785607',
  '990',
  '롯데캐슬클래식',
  '15'],
 ['1150010500',
  '아파트',
  '253',
  '서울특별시 강서구 마곡동 422 마곡신안',
  'A15721003',
  '253',
  '마곡신안',
  '3'],
 ['1156011700',
  '아파트',
  '348',
  '서울특별시 영등포구 당산동 374 래미안당산1차아파트',
  'A15081001',
  '348',
  '래미안당산1차아파트',
  '4'],
 ['1121510500',
  '아파트',
  '178',
  '서울특별시 광진구 자양동 786 자양현대6차',
  'A14319301',
  '17

In [ ]:
'''
# 아파트 정보 csv 파일 저장
result = pd.DataFrame(rowList, columns=columnList)
result.to_csv('aptInfo.csv', index=False)
'''

In [3]:
# 아파트 정보 불러와서 확인
df = pd.read_csv('../dataSet/aptInfo.csv')
df

,bjdCode,codeAptNm,hoCnt,kaptAddr,kaptCode,kaptdaCnt,kaptName,kaptDongCnt
0,1154510300,아파트,233.0,서울특별시 금천구 시흥동 173-8 시흥성지,A15303103,233.0,시흥성지,2.0
1,1168011800,아파트,321.0,서울특별시 강남구 도곡동 543-7 도곡1차아이파크,A13527007,321.0,도곡1차아이파크,5.0
2,1147010100,아파트,186.0,서울특별시 양천구 신정동 1198-3 신정대성유니드아파트2차,A15886402,186.0,신정대성유니드아파트2차,2.0
3,1135010600,아파트,960.0,서울특별시 노원구 중계동 366 중계라이프신동아청구아파트,A13986111,960.0,중계라이프신동아청구아파트,17.0
4,1135010400,아파트,730.0,서울특별시 노원구 하계동 288 하계2차현대아파트,A13923106,730.0,하계2차현대아파트,12.0
...,...,...,...,...,...,...,...,...
2791,1174010200,주상복합,656.0,서울특별시 강동구 고덕동 210-1 고덕센트럴푸르지오,A10024562,656.0,고덕센트럴푸르지오,4.0
2792,1156010200,도시형 생활주택(주상복합),185.0,서울특별시 영등포구 영등포동1가 94-2 여의도더리브스타일,A10024490,185.0,여의도더리브스타일,1.0
2793,1120011500,아파트,114.0,서울특별시 성동구 성수동2가 840 성수 대명루첸아파트,A10025101,114.0,성수 대명루첸아파트,3.0
2794,1144011100,아파트,1061.0,서울특별시 마포구 신수동 93-102 신촌숲 아이파크 아파트,A10024974,1015.0,신촌숲 아이파크 아파트,7.0
